In [ ]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from nltk.tokenize import wordpunct_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, clone_and_build_model
from keras.layers import Input, Embedding, LSTM, Dense
from transformers import pipeline
import tensorflow as tf
from tensorflow import keras

# Problem Statement

Our main research question is: **Can we use the text of user reviews to predict the star ratings for reviews of businesses in Philadelphia?**

To answer this question, we'll try to build a model that takes a Yelp review and predicts the star rating. Then, to see whether star ratings accurately match sentiment analysis, we will compare both the true star rating labels and our predicted star ratings to the sentiment analysis results of a pretrained model from Hugging Face.

# Exploratory Data Analysis

## Loading Data
The initial dataset was quite large (even when we limited our focus to only Philadelphia). Thus, we had to split the `.csv` files up, which is why we have this slightly clunky import code. 

In [ ]:
# load in business data
business = pd.read_json('data/yelp_academic_dataset_business.json', lines=True)
# keep only those businesses in the city with the most observations (Philadelphia)
# and only businesses which are restaurants
max_city = business['city'].value_counts().idxmax()
business = business.loc[(business['city'] == max_city) & (business['categories']).str.contains('Restaurants')]

In [ ]:
# load in review data from separate csv's located in data folder
review = []
for i in range(1, 8):
    review.append(pd.read_csv(f'data/review{i}.csv'))
review = pd.concat(review)

### Inspecting `business`

We don't really have to worry about missing values for our `business` dataset. The only featurse with any missing values are `hours` and `attributes` (itself a dictionary of binary categories which describe the business, such as "nice ambiance" or "good views") which we are not planning on incorporating into our model. 

In [ ]:
business.isnull().sum()/len(business.index)

In [ ]:
business.describe()

It seems that four- and five-star reviews are more common than lower ratings, which later motivates our decision to consider four and five star reviews to be positive, while considering one to three star reviews to be negative. 

In [ ]:
plt.hist(business['stars'], bins=np.arange(0.75, 5.75, 0.5), edgecolor="white")
plt.xlabel("Stars")
plt.ylabel("Count")
plt.title("Frequency of Star Values for Restaurants")
plt.show()

The distribution of review counts in the strip plot below closely mirrors the histogram showing frequency of star values. For example, four star reviews appear the most in the dataset, and restaurants with an average of four stars often have the most reviews attributed to them. On the other hand, not many restaurants have an average of one or five stars, and those with low ratings don’t have many reviews. This suggests that when more reviews are left for a restaurant, the average rating starts to converge toward four stars.

In [ ]:
sns.stripplot(data=business, x='stars', y='review_count', alpha = 0.2, jitter=0.3)
plt.ylim(-10, 2000)
plt.xlabel("Average Star Rating")
plt.ylabel("Review Count")
plt.title("Review Counts for Restaurants across Average Star Rating");

### Inspecting `reviews`

In [ ]:
review.isnull().sum()/len(review.index)

In [ ]:
review.describe()

The distribution of stars for reviews shows a different pattern than the distribution of stars for restaurants. The plurality of reviews are five stars and there are more one-star reviews than two-star reviews, but when the reviews are averaged for each restaurant, there is a regression toward the mean effect.

In [ ]:
plt.hist(review['stars'], bins=np.arange(0.5, 6.5, 1), edgecolor="white")
plt.xlabel("Stars")
plt.ylabel("Count")
plt.title("Frequency of Star Values for Reviews")
plt.show()

Most reviews are short and the number of reviews of a given length generally decreases as the length increases, but there are more 4500-5000 character reviews than there are 4000-4500 character reviews. The boxplots indicate that reviews for businesses with two or three stars are slightly longer while reviews for businesses with five stars tend to be shorter, but generally reviews of all star ratings follow a very similar distribution. Additionally, for all star ratings, there are many outliers with long review lengths. Because most of the reviews are shorter, when pre-processing the reviews text we should be able to truncate longer reviews without too much negative impact on model performance.

In [ ]:
plt.hist(review['text'].str.len(), edgecolor="white")
plt.xlabel('Review Length (Characters)')
plt.ylabel('Count')
plt.yscale('log')
plt.title('Histogram of Review Lengths')
plt.show()

In [ ]:
sns.boxplot(x=review['stars'], y=review['text'].str.len())
plt.xlabel("Stars")
plt.ylabel("Review Length (Characters)")
plt.title("Distribution of Review Length by Star Rating");

# Text Processing

In order to prepare `reviews`, we simply use `text` as our input variable and `stars` as our output variable. We then save the arrays for future use. However, in order to feed `text` into a neural network, we first had to tokenize it -- this process took a long time, so the code is commented out (so that it is not accidentally run). We utilized `wordpunct_tokenize()` in order to pre-process the text. Additionally, because we chose to pad the sequences, we started counting our word to index dictionary at 1. 

In [ ]:
def clean(string):
    return wordpunct_tokenize(string.lower())

review['text'] = review['text'].apply(clean)
words = review['text'].apply(set)
words = set().union(*words)
len(words)

In [ ]:
word2idx = {w: i + 1 for i, w in enumerate(words)}
f = open("data/word2idx.pkl", "wb")
pickle.dump(word2idx, f)
f.close()
X = review['text'].apply(lambda r: [word2idx[w] for w in r])
X

In [ ]:
max_words = max([len(i) for i in X])
X = pad_sequences(X, max_words)
X.shape

In [ ]:
y = review['stars'].to_numpy()
np.savez_compressed('data/reviews.npz', X=X, y=y)

# Baseline Models

First, we load in the data from `reviews.npz` and use `tensorflow` `Dataset`.

In [ ]:
# for quick loading
reviews = np.load('data/reviews.npz')
X, y = reviews['X'], reviews['y'] - 1
with open('data/word2idx.pkl', 'rb') as f:
    word2idx = pickle.load(f)

## train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=109)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=109)

## check shapes
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

In [ ]:
## Create data pipeline
batch_size = 1024
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val)).shuffle(len(X_val)).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test)).shuffle(len(X_test)).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

## Naive Feed Forward Network

This is the most naive approach to the problem. Given the complexity of the text, we didn't expect a feed forward neural network to perform very well, specifically because it would have issues remembering context (given that it has no recurrent relationships between different nodes).

In [ ]:
ff1 = Sequential(name='Naive_FFNN')
ff1.add(Dense(64, activation='relu',input_dim=X_train.shape[1]))
ff1.add(Dense(5, activation='softmax'))
ff1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
ff1.summary()

In [ ]:
epochs=10
ff1.fit(train_data, validation_data=val_data, epochs=epochs)

As expected, this model did not perform very well. While we only trained for 10 epochs, we could see that the model accuracy on the validation set would not get much higher than 40%.

In [ ]:
plt.plot(range(1, 11), ff1.history.history['loss'], label='train')
plt.plot(range(1, 11), ff1.history.history['val_loss'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Categorical Cross Entropy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot(range(1, 11), ff1.history.history['accuracy'], label='train')
plt.plot(range(1, 11), ff1.history.history['val_accuracy'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

### Evaluation

In [ ]:
ff1.evaluate(test_data)

In [ ]:
y_pred = np.argmax(ff1.predict(X_test, batch_size = 1024), axis = -1).flatten() 

# Creating a confusion matrix, which compares the y_test and y_pred
cm = confusion_matrix(y_test, y_pred) 
cm_df = pd.DataFrame(cm, index = [1, 2, 3, 4, 5], columns = [1, 2, 3, 4, 5]) 
# Plotting the confusion matrix
plt.figure(figsize=(5,4)) 
sns.heatmap(cm_df, annot=True) 
plt.title('Confusion Matrix')
plt.ylabel('Actal Values') 
plt.xlabel('Predicted Values') 
plt.show()

### Downsampling

After training our baseline model, a Feed Forward Neural Network, we observed that it predicted 5 stars for almost every review because there was an over-concentration of 5-star reviews in the dataset from Yelp. This shows that the class imbalance we identified in EDA has a large negative impact on model performance. Therefore, we used downsampling to fix this for the LSTM (our first-draft of a final model). We resampled reviews for the training and validation data, restricting our downsampled data to contain 37,000 reviews from each star-class for training and around 9,000 for the validation data. This is because 2-star reviews were the least frequent class of reviews in our histograms and there were about 50,000 of them before doing the train-test split. We did not perform downsampling on the test data so that we can evaluate the model on data that accurately reflects real world data.

In [ ]:
X_d_train = []
y_d_train = []

X_d_val = []
y_d_val = []

_, train_counts = np.unique(y_train, return_counts = True)
smallest_train = min(train_counts)
_, val_counts = np.unique(y_val, return_counts = True)
smallest_val = min(val_counts)

for i in range(5):
    X_di_train, y_di_train = resample(X_train[y_train == i], y_train[y_train == i], 
                                      random_state = 109, 
                                      n_samples = smallest_train, 
                                      replace = False)
    X_di_val, y_di_val = resample(X_val[y_val == i], y_val[y_val == i], 
                                  random_state = 109,
                                  n_samples = smallest_val,
                                  replace = False)
    X_d_train.append(X_di_train)
    y_d_train.append(y_di_train)
    
    X_d_val.append(X_di_val)
    y_d_val.append(y_di_val)
    
X_d_train = np.stack(X_d_train).reshape(smallest_train * 5, X.shape[1])
y_d_train = np.stack(y_d_train).flatten()

X_d_val = np.stack(X_d_val).reshape(smallest_val * 5, X.shape[1])
y_d_val = np.stack(y_d_val).flatten()

X_d_train.shape, y_d_train.shape, X_d_val.shape, y_d_val.shape

In [ ]:
plt.hist(y_d_val, bins=np.linspace(-0.5, 4.5, 6), rwidth=0.7)
plt.xticks(range(5), range(1, 6))
plt.xlabel('Star Rating')
plt.ylabel('Frequency')
plt.title('Histogram of Star Ratings in Downsampled Validation Set')
plt.show()

In [ ]:
d_train_data = tf.data.Dataset.from_tensor_slices((X_d_train, y_d_train)).shuffle(len(X_d_train)).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
d_val_data = tf.data.Dataset.from_tensor_slices((X_d_val, y_d_val)).shuffle(len(X_d_val)).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

### Training FFNN on Downsampled Data

In [ ]:
ff2 = tf.keras.models.clone_model(ff1)
ff2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
ff2.summary()

In [ ]:
ff2.fit(d_train_data, validation_data=d_val_data, epochs=epochs)

In [ ]:
y_pred = np.argmax(ff2.predict(X_test, batch_size = 1024), axis = -1).flatten() 

# Creating a confusion matrix, which compares the y_test and y_pred
cm = confusion_matrix(y_test, y_pred) 
cm_df = pd.DataFrame(cm, index = [1, 2, 3, 4, 5], columns = [1, 2, 3, 4, 5]) 
# Plotting the confusion matrix
plt.figure(figsize=(5,4)) 
sns.heatmap(cm_df, annot=True) 
plt.title('Confusion Matrix')
plt.ylabel('Actal Values') 
plt.xlabel('Predicted Values') 
plt.show()

## LSTM

For our final model, we decided to use LSTM. Given that our naive model performed poorly, even after downsampling, we decided that a language model was necessary in order to get the level of test accuracy we desired (at least >75%). In the initial draft of our final model, we were able to achieve training accuracy of 75% and testing accuracy of 60% on our dataset after around 10 epochs. However, we found that our model suffered from overfitting (after around 5 epochs, the validation loss began to rise quickly). 

In future iterations of this model, we hope to add ELMo embeddings. This is better than the standard Keras Embedding layer (word2vec) that we used in first training the LSTM model because ELMo works bidirectionally (considering semantic contexts from before and after a word in a sentence) whereas word2vec only works in one direction. Further, ELMo embeddings are created on a character-basis–not on a word basis–so that our model can learn contexts of words it hasn’t yet encountered. This could be useful in our context because all of the data has been generated by people, prone to errors like spelling mistakes. The ELMo embeddings will let us reflect contexts of mis-spelled/erroneous words.

In [ ]:
inputs = Input(shape=(X_d_train.shape[1],))
n = Embedding(len(word2idx) + 1, 32, mask_zero=True)(inputs)
n = LSTM(32)(n)
outputs = Dense(5, activation='softmax')(n)
model2 = Model(inputs=inputs, outputs=outputs, name="LSTM")
model2.summary()
model2.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
model2.fit(train_data, validation_data=val_data, epochs=epochs)

In [ ]:
plt.plot(range(1, 11), model2.history.history['loss'], label='train')
plt.plot(range(1, 11), model2.history.history['val_loss'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Categorical Cross Entropy')
plt.legend()
plt.grid()
plt.show()

### Evaluation
In order to test this model on new input, we created a simple function which tokenized and padded sample reviews. It seems that our model is fairly close to predicting stars in the same way we would (when we use `argmax` to find the class with the highest probability). In particular, the model is good at predicting the extreme negativity expressed in the final review we tested. 

In [ ]:
def as_input(string):
    return pad_sequences(np.array([[word2idx[w] for w in wordpunct_tokenize(string.lower())]]), X.shape[1])

In [ ]:
model2.predict(as_input('This food is so bad'))

In [ ]:
model2.predict(as_input('This food is so good'))

In [ ]:
model2.predict(as_input('One of the worst places Ive been, unfortunately! Rude staff, horrible service and tasteless food and coffee. Would never come back or recommend this place.'))

# Sentiment Analysis with Hugging Face Model

Based on the work we did in the most recent lab, we would also like to expand our project to include a sentiment analysis. To classify the sentiment of the reviews, we will use a pipeline from Hugging Face. The code below shows how the sentiment analysis pipeline performs on the first five reviews in our dataset, and it seems to be fairly accurate even without any fine tuning.

As noted in our previous milestone, we thought that we could map four- and five-star reviews into a “positive” category, while one- through three-star reviews would be “negative.” We plan to compare sentiments to users’ star reviews by creating a stacked bar chart: Each bar will represent a number of stars, and the stacks will be based on the number of positive and negative reviews in each star category. This should show us whether users deemed to have written negative reviews actually assign one-three stars to those businesses, or if they are perhaps too sympathetic and leave four-five stars despite writing bad reviews. We expect the results of the three-star bar to be most interesting because it’s plausible that some users associate three stars with positivity/neutrality while others associate it with negativity.


In [ ]:
df = pd.read_csv("data/review1.csv")

In [ ]:
cls = pipeline('sentiment-analysis')

In [ ]:
for i in range(5):
  print(df['text'][i])
  print()

In [ ]:
# Need to truncate data to be able to predict on all reviews
cls(list(df['text'][0:5]))